In [3]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
train_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

test_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.ToTensor(),
])


In [5]:
train_data = datasets.ImageFolder(root="data/train", transform=train_transform)
train = DataLoader(train_data, batch_size = 32, shuffle = True)

In [6]:
test_data = datasets.ImageFolder(root="data/validation", transform=test_transform)
test = DataLoader(test_data, batch_size = 32, shuffle = False)

In [12]:
class CNNBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, conv_kernel, conv_stride, conv_padding, pool_kernel, pool_stride):
        super().__init__()
        L = [
            torch.nn.BatchNorm2d(in_channels),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels, out_channels, conv_kernel, conv_stride, conv_padding, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.SiLU(),
            torch.nn.MaxPool2d(pool_kernel, pool_stride),
        ]
        self.network = torch.nn.Sequential(*L)
    
    def forward(self, x):
        return self.network(x)

class CNNClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        in_channels = 3
        out_channels = 32
        num_classes = 2
        kernel_size = 7
        stride = 1
        padding = 3
        L = [
            CNNBlock(in_channels, out_channels, kernel_size, stride, padding, 2, 2),
            torch.nn.Dropout(p=0.05),
            CNNBlock(32, 64, 3, 1, 1, 2, 2),
            torch.nn.Dropout(p=0.05),
            CNNBlock(64, 128, 3, 1, 1, 2, 2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(32768, num_classes)
        ]
        self.network = torch.nn.Sequential(*L)
        self.transforms = torchvision.transforms.Compose([torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def forward(self, x):
        x = self.transforms(x)
        return self.network(x)

In [13]:
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [20]:
def train_iter(epoch, net, trainDataLoader, optimizer, criterion):
    net.train()
    total = 0
    train_loss = 0
    correct = 0

    for inputs, targets in trainDataLoader:
        optimizer.zero_grad()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    avg_loss = train_loss / len(trainDataLoader)
    accuracy = 100 * correct / total
    print('Epoch: {} \tTraining Loss: {:.25f} \tTraining Accuracy: {:.25f}'.format(epoch, avg_loss, accuracy))
    return avg_loss

def train_model(num_epochs, net, trainDataLoader, optimizer, criterion):
  for i in range(num_epochs):
    train_iter(i, net, trainDataLoader, optimizer, criterion)

In [21]:
train_model(5, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 0.5145394382279482314856978 	Accuracy: 83.5625000000000000000000000
Epoch: 1 	Training Loss: 0.5330764148568800653293920 	Accuracy: 83.3125000000000000000000000
Epoch: 2 	Training Loss: 0.5777742280856545953326986 	Accuracy: 83.0625000000000000000000000
Epoch: 3 	Training Loss: 0.9286311275804763898378269 	Accuracy: 78.6250000000000000000000000
Epoch: 4 	Training Loss: 0.4577076206594934415505804 	Accuracy: 86.2500000000000000000000000


In [24]:
def eval_model(net, testDataLoader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testDataLoader:
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
    print('Accuracy of the network on the set of %d test images: %d %%' % (len(test),
        100 * correct / total))

In [25]:
eval_model(model, test)

Accuracy of the network on the set of 400 test images: 61 %
